In [90]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import requests
import os
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()

True

### Two Types of investments
####    Long Term
####    Short Term

In [91]:
# one Exchange-Traded Fund (ETF) representing a broad market index, 
#   QQQ
#   SPY
# one index mutual fund for diversified exposure to the overall market

# one Forex pair to hedge against currency risk.

# Split them into Long Term and Short Term 25%
# QQQ
# SPY
# AGI GOLD
# MARA Energy
# NVDA 

In [92]:
# Database
# Portfolio (Net value, cash balance, Realized profit, Unrealized profit)
# Investement (ID, Type, #ofShares, Avg Cost, mrkt value)
# Buy/Sell (investement_ID, #ofShares, Date)
# candle_sticks(investement_ID, date, closePrice)

In [93]:

def polygon(stock_ticker):
        
    polygon_url = 'https://api.polygon.io/v2/aggs/ticker'
    stocksTicker = f'/{stock_ticker}'
    timespan = '/range/1/day'
    date_from = '/2023-01-01'
    date_to = '/2024-07-03'
    params = '?adjusted=true&sort=asc&'
    polygon_key = os.environ['polygon_api_key']

    params = {'adjusted':'true', 
            'sort': 'asc',
            'apiKey': polygon_key
            }

    full_url = polygon_url + stocksTicker + timespan + date_from + date_to

    response2 = requests.get(full_url, params)

    polygon_data = response2.json()

    # Access the "results" part
    results = polygon_data["results"]
    
    df_polygon = pd.DataFrame(results)
    df_polygon.head(5)

    # Convert the Unix timestamp to datetime
    df_polygon['date'] = pd.to_datetime(df_polygon['t'], unit='ms')

    # Format the datetime to the desired string format (YYYY-MM-DD)
    df_polygon['date'] = df_polygon['date'].dt.strftime('%Y-%m-%d')

    # Drop the unix time column
    df_polygon = df_polygon.drop(columns=['t'])

    df_polygon.rename(columns={'v': 'volume', 'o':'open', 'c':'close', 'h':'high', 'l':'low'}, inplace=True)

    df_polygon = df_polygon[['date', 'open', 'high', 'low', 'close', 'volume', 'vw', 'n']]
    
    return df_polygon


In [94]:
def alpha_vantage(stock_ticker):

    base_url = 'https://www.alphavantage.co'
    stocks_endpoint = '/query'

    stocks_url = base_url+stocks_endpoint

    api_key = os.environ['alphavantage_api_key']

    params = {'function':'TIME_SERIES_DAILY', 
            'symbol': stock_ticker,
            'outputsize': 'full',
            'apikey' : api_key}

    alpha_response = requests.get(stocks_url, params)
    alpha_data = alpha_response.json()
    time_series = alpha_data["Time Series (Daily)"]

    # Filter the entries to only include dates from 01/01/2023 onwards
    start_date = datetime.strptime("2023-01-01", "%Y-%m-%d")
    filtered_data = {date: values for date, values in time_series.items() if datetime.strptime(date, "%Y-%m-%d") >= start_date}
    
    df_alpha = pd.DataFrame.from_dict(filtered_data, orient="index")
    df_alpha.index = pd.to_datetime(df_alpha.index)
    # Rename columns to remove the numeric prefix
    df_alpha = df_alpha.rename(columns=lambda x: x.split(". ")[1])  

    # Reset the index to create a new numerical index
    df_alpha = df_alpha.reset_index()
    df_alpha.rename(columns={'index': 'date'}, inplace=True)

    # Sort DataFrame by 'date' column in ascending order
    df_alpha = df_alpha.sort_values(by='date', ascending=True)
    df_alpha = df_alpha.reset_index()

    df_alpha = df_alpha.drop(columns=['index'])

    df_alpha.head(5)

    return df_alpha
 

In [99]:

df_QQQ = polygon('SPY')
df_QQQ.head(5)

,date,open,high,low,close,volume,vw,n
0,2023-01-03,384.37,386.43,377.8310,380.82,74850731.0,380.9576,590240
1,2023-01-04,383.18,385.88,380.0000,383.76,85934098.0,383.1494,632808
2,2023-01-05,381.72,381.84,378.7600,379.38,76275354.0,380.2625,530896
3,2023-01-06,382.61,389.25,379.4127,388.08,104052662.0,385.2463,687390
4,2023-01-09,390.37,393.70,387.6700,387.86,73978071.0,390.3628,549428


In [100]:
df_QQQ2 = alpha_vantage('SPY')
df_QQQ2.head(5)

,date,open,high,low,close,volume
0,2023-01-03,384.3700,386.4300,377.8310,380.8200,74850731
1,2023-01-04,383.1800,385.8800,380.0000,383.7600,85934098
2,2023-01-05,381.7200,381.8400,378.7600,379.3800,76970456
3,2023-01-06,382.6100,389.2500,379.4127,388.0800,104189603
4,2023-01-09,390.3700,393.7000,387.6700,387.8600,73978071
